## DarkSky APi to InfluxDB

In [52]:
import pandas as pd

In [53]:
coords = (45.1973288, 5.7139923)
day = '2017-10-06T00:00:00'

In [54]:
from darksky import forecast

# Load the API key for darksky
with open('darksky_key.txt') as f:
    KEY = f.read()

EXCLUDE = ['currently', 'minutely', 'daily', 'flags']  # from the query

In [55]:
# query
data = forecast(KEY, *coords, units='si', lang='fr', time=day, exclude=EXCLUDE)

In [56]:
# build DF from recots (dict)
records = data['hourly']['data']  
weatherdataDF = pd.DataFrame.from_records(records, index='time')

weatherdataDF.index = pd.to_datetime(weatherdataDF.index, unit='s')

In [57]:
weatherdataDF

,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipIntensity,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
time,,,,,,,,,,,,,,,,,
2017-10-05 22:00:00,13.57,0.18,7.40,0.66,clear-night,255.96,0.0000,0.00,NaN,1019.09,Dégagé,13.57,0,10.01,10,2.15,0.97
2017-10-05 23:00:00,13.67,0.18,8.02,0.69,clear-night,253.89,0.0000,0.00,NaN,1018.90,Dégagé,13.67,0,10.01,6,2.23,0.91
2017-10-06 00:00:00,15.27,0.29,7.18,0.59,partly-cloudy-night,252.64,0.0000,0.00,NaN,1018.10,Nuages Épars,15.27,0,10.01,1,1.79,1.02
2017-10-06 01:00:00,15.49,0.40,7.62,0.59,partly-cloudy-night,252.59,0.0152,0.03,rain,1017.77,Nuages Épars,15.49,0,10.01,359,2.41,1.12
2017-10-06 02:00:00,15.02,0.49,8.21,0.64,partly-cloudy-night,253.09,0.0787,0.07,rain,1017.44,Nuages Épars,15.02,0,10.01,358,3.29,1.24
2017-10-06 03:00:00,14.84,0.75,9.08,0.68,rain,253.77,0.1549,0.11,rain,1017.21,Bruine,14.84,0,10.01,358,4.05,1.35
2017-10-06 04:00:00,13.81,0.82,9.72,0.76,rain,254.50,0.1880,0.13,rain,1017.09,Bruine,13.81,0,6.00,355,4.58,1.48
2017-10-06 05:00:00,13.54,0.71,10.36,0.81,rain,255.58,0.1854,0.13,rain,1017.04,Bruine,13.54,0,10.01,353,4.97,1.64
2017-10-06 06:00:00,13.23,0.67,10.41,0.83,rain,256.53,0.1549,0.11,rain,1017.09,Bruine,13.23,0,10.01,355,5.39,1.89


In [60]:
from influxdb import DataFrameClient

In [61]:
# see https://influxdb-python.readthedocs.io/en/latest/examples.html#tutorials-pandas
"""Instantiate the connection to the InfluxDB client."""

host, port = 'localhost', 8086
user, password = 'root', 'root'
dbname = 'weatherdata'
protocol = 'line'

client = DataFrameClient(host, port, user, password, dbname)


print("Create database: " + dbname)
client.create_database(dbname)

print("Write DataFrame")

measurement_name = 'darksky'
client.write_points( weatherdataDF, measurement_name, {'location': 'gre'}, protocol=protocol)


Create database: weatherdata
Write DataFrame


True